# Elastic search powered skatte-ABC search engine

In [15]:
import json, os

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.elasticsearch import ElasticsearchStore
from llama_index.core import SimpleDirectoryReader

from dotenv import load_dotenv

INDEX_NAME = "index_skatt_test"


## Vector store

In [12]:
from llama_index.vector_stores.elasticsearch import ElasticsearchStore

vector_store = ElasticsearchStore(
    index_name=INDEX_NAME, es_url="http://localhost:9200/"
)

## Data ingestion

In [16]:
documents=SimpleDirectoryReader("../../data/test/").load_data()

In [17]:
ollama_embedding = OllamaEmbedding("mistral")

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=2000, chunk_overlap=500),
        ollama_embedding,
    ],
    vector_store=vector_store,
)

pipeline.run(documents=documents)

[TextNode(id_='d011aba1-8267-4460-bcad-c47d0ffa8927', embedding=[3.137472629547119, 2.923325538635254, -2.1294937133789062, -7.410823345184326, 3.9856507778167725, -6.628032207489014, -11.123067855834961, 10.434712409973145, 0.09959205985069275, -1.0423760414123535, -6.626099109649658, 0.8047257661819458, 2.8421175479888916, 0.7271662354469299, -3.2760093212127686, 4.341631889343262, 1.6887588500976562, 0.302115261554718, -0.794970691204071, -1.1060103178024292, 4.34724760055542, -3.8377487659454346, -1.662314534187317, 6.8955841064453125, -2.0501794815063477, -4.713207721710205, -1.4355967044830322, -2.1279494762420654, -2.3075718879699707, 1.6712455749511719, -0.6940759420394897, 4.599970817565918, -2.1958327293395996, -2.122361183166504, 0.7140645980834961, 3.120875835418701, -10.65142822265625, 1.8720529079437256, 6.9918532371521, 4.823596000671387, 5.262054920196533, -3.050100088119507, -7.095854759216309, 4.117349624633789, 3.2143185138702393, 2.5481579303741455, -4.6823201179504

## Querying

In [ ]:
from llama_index.core import VectorStoreIndex, QueryBundle, Response, Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from index import es_vector_store

# Local LLM to send user query to
local_llm = Ollama(model="mistral")
Settings.embed_model= OllamaEmbedding("mistral")

index = VectorStoreIndex.from_vector_store(vector_store)
query_engine = index.as_query_engine(local_llm, similarity_top_k=5)

In [ ]:
query="Give me summary of water related issues"
bundle = QueryBundle(query, embedding=Settings.embed_model.get_query_embedding(query))
result = query_engine.query(bundle)